In [ ]:
%pip install pyodbc

In [ ]:
%pip install plotly==5.6.0

In [ ]:
import pyodbc
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
conn = pyodbc.connect('#see pyodbc documentation')

In [ ]:
#think before running (download time > 6 min)
cursor = conn.cursor()
query = "SELECT * FROM table"
df = pd.read_sql(query, conn)

In [ ]:
#df.to_csv('year_to_date.csv')
#df = pd.read_csv('year_to_date.csv')

In [ ]:
df_opp = pd.read_excel('OPP_unique.xlsx')

In [ ]:
df_opp['ArtNr'] = df_opp['ArtNr'].astype('int64')

In [ ]:
df2 = df[(df['DokDatum'] > '2021-01-01') & (df['DokDatum'] < '2022-03-31')]
df2['DokDatum'] = pd.to_datetime(df2['DokDatum'], format = '%Y-%m-%d')
df2['YYYY_MM'] = pd.to_datetime(df2['DokDatum']).dt.strftime('%Y-%m')

In [ ]:
df3 = df2[df2['DokStufe'] == '4']

In [ ]:
df4 = df3[df3['Artikelnummer'].str.isnumeric()]
df6 = df3[df3['Artikelnummer'].str.isnumeric() == False]

In [ ]:
df4['Artikelnummer'] = df4['Artikelnummer'].astype('int64')

In [ ]:
df5 = df4.merge(df_opp, how = 'left', left_on = 'Artikelnummer', right_on = 'ArtNr')

In [ ]:
df5_opp = df4.merge(df_opp, how = 'inner', left_on = 'Artikelnummer', right_on = 'ArtNr')

In [ ]:
df5['DokDatum'] = pd.to_datetime(df5['DokDatum'], format = '%Y-%m-%d')

In [ ]:
df7 = df5.append(df6)

In [ ]:
df7['OPP'] = df7['OPP'].fillna('Non_OPP')

In [ ]:
df7_non_OPP = df7[df7['OPP'] =='Non_OPP']

In [ ]:
df8 = pd.pivot_table(df7,index=['DokADSachbearbeiterName', 'OPP'], values = ['UmsatzMIs', 'DBCHFMIS'], aggfunc = np.sum)

In [ ]:
df8['DB%'] = df8['DBCHFMIS'] / df8['UmsatzMIs'] 

In [ ]:
df8.to_excel('aussendienst_opp_analysis.xlsx')

In [ ]:
df9 = pd.pivot_table(df5_opp,index=['Artikelnummer', 'Art.Bezeichnung'], values = ['UmsatzMIs', 'DBCHFMIS'], aggfunc = np.sum)

In [ ]:
df9['DB%'] = df9['DBCHFMIS'] / df9['UmsatzMIs']

In [ ]:
fig = px.scatter(df_9_non_negative, x = 'UmsatzMIs', y = "DB%",log_x = True)
fig.show()

In [ ]:
df10 = pd.pivot_table(df5_opp,index=['YYYY_MM'], values = ['UmsatzMIs', 'DBCHFMIS'], aggfunc = np.sum)
df10['DB%'] = df10['DBCHFMIS'] / df10['UmsatzMIs']

In [ ]:
df11 = pd.pivot_table(df7,index=['YYYY_MM'], values = ['UmsatzMIs', 'DBCHFMIS'], aggfunc = np.sum)
df11['DB%'] = df11['DBCHFMIS'] / df11['UmsatzMIs']

In [ ]:
df12 = pd.pivot_table(df7_non_OPP,index=['YYYY_MM'], values = ['UmsatzMIs', 'DBCHFMIS'], aggfunc = np.sum)
df12['DB%'] = df12['DBCHFMIS'] / df12['UmsatzMIs']
df12['% Total Umsatz'] = df12['UmsatzMIs'] / df11['UmsatzMIs']

In [ ]:
df10['% Total Umsatz'] = df10['UmsatzMIs'] / df11['UmsatzMIs']

In [ ]:
df10 = df10.rename(columns = {'DB%': 'DB% (OPP)', '% Total Umsatz': '% Total Umsatz (OPP)'})
df12 = df12.rename(columns = {'DB%': 'DB% (non_OPP)', '% Total Umsatz': '% Total Umsatz (non_OPP)'})
df11 = df11.rename(columns = {'DB%': 'DB% (total)'})

In [ ]:
df13 = pd.concat([df10, df12, df11], axis = 1)
df13 = df13[['DB% (OPP)', 'DB% (non_OPP)', 'DB% (total)', '% Total Umsatz (OPP)', '% Total Umsatz (non_OPP)']]

In [ ]:
df13.to_excel(db_opp_analysis.xlsx)

In [ ]:
df14 = df13.reset_index()
fig2 = make_subplots(specs = [[{'secondary_y': False}]])
fig2.add_trace(
    go.Scatter(x = df14['YYYY_MM'], y = df14['DB% (OPP)'], name = 'DB% (OPP)')
)
fig2.add_trace(
    go.Scatter(x = df14['YYYY_MM'], y = df14['DB% (non_OPP)'], name = 'DB% (non_OPP)')
)
fig2.add_trace(
    go.Scatter(x = df14['YYYY_MM'], y = df14['DB% (total)'], name = 'DB% (total)')
)
fig2.show()